# Harmonization of MIDRC Imaging Study Descriptions using the LOINC Playbook
---
by Chris Meyer, PhD

Manager of Data and User Services at the Center for Translational Data Science at University of Chicago

November 2022

---
This Jupyter notebook demonstrates how to map the imaging_study node's `study_description` and `modality` properties to a `loinc_code` by utilizing the MIDRC DQH committee's [LOINC mapping table in GitHub](https://github.com/MIDRC/midrc_dicom_harmonization/tree/main/out). Once the imaging study has been mapped to a LOINC code, the code can be used to derive other LOINC properties. The following [LOINC properties](https://github.com/uc-cdis/midrc_dictionary/blob/deae581f0fb8b9ae5add1458d7882e189ba97af6/gdcdictionary/schemas/imaging_study.yaml#L77) are on the `imaging_study` node of the [MIDRC data dictionary](https://data.midrc.org/dd):
* loinc_code
* loinc_long_common_name
* loinc_method
* loinc_system
* loinc_contrast

In [1]:
# Import Python Packages and scripts
import pandas as pd
import numpy as np
from pathlib import Path
import sys, os, copy, datetime, shutil


import gen3
from gen3.submission import Gen3Submission
from gen3.auth import Gen3Auth
from gen3.index import Gen3Index
from gen3.query import Gen3Query


In [2]:
# download and import some custom Python scripts from https://github.com/cgmeyer/gen3sdk-python
loinc_dir = "/Users/christopher/Documents/Notes/MIDRC/LOINC" #set your working dir
os.chdir(loinc_dir)
os.system("wget https://raw.githubusercontent.com/cgmeyer/gen3sdk-python/master/expansion/expansion.py -O expansion.py")
from expansion import Gen3Expansion


In [3]:
# Initiate instances of the Gen3 SDK Classes using credentials file downloaded from https://staging.midrc.org/identity
# You can view the SDK code/functions in GitHub: https://github.com/uc-cdis/gen3sdk-python
sapi = 'https://staging.midrc.org'
scred = '/Users/christopher/Downloads/midrc-staging-credentials.json'
sauth = Gen3Auth(sapi, refresh_file=scred) # authentication class
ssub = Gen3Submission(sapi, sauth) # submission class
squery = Gen3Query(sauth) # query class
sexp = Gen3Expansion(sapi,sauth,ssub) # class with some custom scripts
sexp.get_project_ids()


Getting all project_ids you have access to in the data commons.
['Open-A1', 'Open-A1_PETAL_REDCORAL', 'Open-R1', 'TCIA-COVID-19-AR', 'TCIA-COVID-19-NY-SBU', 'TCIA-COVID-19_CT_Images', 'TCIA-RICORD']


/Users/christopher/Documents/Notes/MIDRC/LOINC/expansion.py:213: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(res["data"]["project"])


['Open-A1',
 'Open-A1_PETAL_REDCORAL',
 'Open-R1',
 'TCIA-COVID-19-AR',
 'TCIA-COVID-19-NY-SBU',
 'TCIA-COVID-19_CT_Images',
 'TCIA-RICORD']

## Prepare the mapping and filter tables
---
Get the mapping table and the filtering attributes tables from the [MIDRC GitHub repository](https://github.com/MIDRC/midrc_dicom_harmonization/tree/main/out) and reformat them to prepare for the LOINC mapping.

Prior to this, you will need to clone the repo using `git clone git@github.com:MIDRC/midrc_dicom_harmonization.git`


In [4]:
## Set the directory to your copy of the GitHub repo and pull main branch to pull latest updates from GitHub
git_dir = "/Users/christopher/Documents/GitHub/MIDRC/midrc_dicom_harmonization/"
os.chdir(git_dir)
os.system("git checkout main")
os.system("git pull origin main")
os.chdir(loinc_dir)


In [5]:
## Create the output/working directory
now = datetime.datetime.now()
today = "{}-{}-{}".format(now.year, now.month, now.day)

results_dir = "{}/results_{}".format(loinc_dir,today)
Path(results_dir).mkdir(parents=True, exist_ok=True)

In [6]:
mapping_file = "{}/out/StudyDescription_mapping_table.csv".format(git_dir)
mapping = pd.read_csv(mapping_file,dtype=str) #['Modality', 'StudyDescription', 'LOINC code', 'L-Long Common Name']

shutil.copy2(mapping_file, results_dir) # copy the mapping table version used for this mapping to the results_dir

mapping.rename(columns={"StudyDescription":"study_description","Modality":"study_modality","LOINC code":"loinc_code"},inplace=True)
mapping.drop(columns=['L-Long Common Name'], inplace=True, errors='ignore')
mapping.drop_duplicates(inplace=True)
mapping['study_description'] = mapping.apply(lambda row: row['study_description'].casefold(),axis=1)
mapping['study_modality'] = mapping.apply(lambda row: row['study_modality'].casefold(),axis=1)

modalities = list(set(mapping['study_modality']))
descriptions = list(set(mapping['study_description']))
codes = list(set(mapping['loinc_code']))

display(mapping)
print("study_modality values in mapping table: {}".format(modalities))
print("Number of LOINC codes in mapping table: {}".format(len(codes)))


,study_modality,study_description,loinc_code
0,ct,[blank],25045-6
1,ct,pet ct fdg imag skull to thigh,81555-5
2,ct,chest pe(adult),79077-4
3,ct,ct chest pulmonary angio with iv con,79077-4
4,ct,ct chest pulmonary embolism (ctpe),79077-4
...,...,...,...
263,us,[blank],25061-3
264,mg,[blank],36625-2
265,nm,[blank],49118-3
266,st,[blank],43526-3


study_modality values in mapping table: ['st', 'mr', 'cr, dx', 'mg', 'ct', '[blank]', 'pt', 'us', 'nm']
Number of LOINC codes in mapping table: 74


In [7]:
filters_file = "{}/out/StudyDescription_filtering_attributes.csv".format(git_dir)
filters = pd.read_csv(filters_file,dtype=str) #['LOINC code', 'L-Long Common Name', 'L-Method', 'L-System', 'Rad.Timing']

shutil.copy2(filters_file, results_dir) # copy the mapping table version used for this mapping to the results_dir

filters.rename(columns= {
    'LOINC code':'loinc_code',
    'L-Long Common Name':'loinc_long_common_name',
    'L-Method':'loinc_method',
    'Rad.Timing':'loinc_contrast',
    'MIDRC-System':'loinc_system'
    },
    inplace=True,
    errors='ignore'
)
filters.drop(columns='L-System',inplace=True,errors='ignore')

codes = list(set(filters['loinc_code']))
print("Number of LOINC codes in filters table: {}".format(len(codes)))
display(filters)

Number of LOINC codes in filters table: 73


,loinc_code,loinc_long_common_name,loinc_method,loinc_contrast,loinc_system
0,18744-3,Bronchoscopy study,Bronchoscopy,NaN,Respiratory system
1,36813-4,CT Abdomen and Pelvis W contrast IV,CT,W,Abdomen
2,42274-1,CT Abdomen and Pelvis WO and W contrast IV,CT,WO & W,Abdomen+Pelvis
3,36952-0,CT Abdomen and Pelvis WO contrast,CT,WO,Abdomen+Pelvis
4,79103-8,CT Abdomen W contrast IV,CT,W,Abdomen
...,...,...,...,...,...
68,36554-4,XR Chest Single view,XR,NaN,Chest
69,83017-4,XR Chest View and Abdomen Supine and Upright,XR,NaN,Chest && Abdomen
70,30745-4,XR Chest Views,XR,NaN,Chest
71,24899-7,XR Ribs Views,XR,NaN,Chest>Ribs


## Get the imaging_study information from MIDRC Staging
---
* Use the [`Gen3Expansion.get_node_tsvs()` SDK function](https://github.com/cgmeyer/gen3sdk-python/blob/389e3945482439ace6e4536e6d0e35c6e48de9c9/expansion/expansion.py#L219) to get all the imaging_studies in MIDRC Staging (staging.midrc.org).

* The function `exp.get_node_tsvs()` will return a master dataframe containing all the imaging_study data from the projects specified.

### In this version of the notebook, we're only getting the TCIA projects: 

* 'TCIA-COVID-19-AR',
* 'TCIA-COVID-19-NY-SBU',
* 'TCIA-COVID-19_CT_Images',
* 'TCIA-RICORD'

In [8]:
os.chdir(loinc_dir)
projects = [ 'TCIA-COVID-19-AR',
 'TCIA-COVID-19-NY-SBU',
 'TCIA-COVID-19_CT_Images',
 'TCIA-RICORD']
#st = sexp.get_node_tsvs(node='imaging_study', overwrite=False, projects=projects,outdir=results_dir)
st = sexp.get_node_tsvs(node='imaging_study', overwrite=True, projects=projects,outdir=results_dir)



Output written to file: /Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-10/imaging_study_tsvs/TCIA-COVID-19-AR_imaging_study.tsv
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-10/imaging_study_tsvs/TCIA-COVID-19-AR_imaging_study.tsv has 256 records.

Output written to file: /Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-10/imaging_study_tsvs/TCIA-COVID-19-NY-SBU_imaging_study.tsv
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-10/imaging_study_tsvs/TCIA-COVID-19-NY-SBU_imaging_study.tsv has 7363 records.

Output written to file: /Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-10/imaging_study_tsvs/TCIA-COVID-19_CT_Images_imaging_study.tsv
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-10/imaging_study_tsvs/TCIA-COVID-19_CT_Images_imaging_study.tsv has 753 records.

Output written to file: /Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-10/imaging_study_tsvs/TCIA-RICORD_imaging_s

In [9]:
dupes = st.loc[st.duplicated(subset='submitter_id',keep=False)].sort_values(by='submitter_id')
print("There are {} duplicate submitter_ids in the list of imaging studies.".format(len(dupes)))
if len(dupes) > 0:
    display(dupes)



There are 0 duplicate submitter_ids in the list of imaging studies.


## Reformat the imaging_study data to prepare for mapping
---
* **NaN values**: In the mapping table, a `study_description` or `study_modality` with a value of `null` or `NaN` is listed as `[blank]`.
* **Case-insensitivity**: we should ignore capitalization of `study_modality` and `study_description` strings when we do the look-up. We'll use `str.casefold()` to ignore case for matching.
* **CR/DX Modality**: in the mapping table, all versions of "CR" and "DX" modalities are denoted "CR, DX"; so, we need to change all occurrences of other spellings to match that exactly.
* **Duplicated study UIDs**: Identify any duplicated study UIDs and remove them from studies.


In [10]:
## Make a copy of the master imaging_study table with only data relevant to LOINC mapping, and change NaN to "[blank]"
studies = copy.deepcopy(st[['submitter_id','study_description','study_modality']])
#studies.index = studies['submitter_id']
studies['study_description'].fillna("[blank]", inplace=True)
studies['study_modality'].fillna("[blank]", inplace=True)

## Make the mapping case-insensitive
studies['study_description'] = studies['study_description'].str.casefold()
studies['study_modality'] = studies['study_modality'].str.casefold()

## Fix any CR/DX study_modality to all be "cr, dx" to match mapping table
studies.replace({"study_modality":{
    'cr':'cr, dx',
    'dx':'cr, dx',
    'cr,dx':'cr, dx',
    'dx,cr':'cr, dx',
    'dx, cr':'cr, dx',
    'crdx':'cr, dx',
    'dxcr':'cr, dx'}},
    inplace=True)

## Identify and drop duplicates
dupes = studies.loc[studies.duplicated(keep=False)]
studies = studies.drop_duplicates()
display(studies)



,submitter_id,study_description,study_modality
0,1.3.6.1.4.1.14519.5.2.1.9999.103.1217405707426...,xr chest ap portable,"cr, dx"
1,1.3.6.1.4.1.14519.5.2.1.9999.103.8220481510500...,xr chest ap portable,"cr, dx"
2,1.3.6.1.4.1.14519.5.2.1.9999.103.1753044623609...,xr chest ap portable,"cr, dx"
3,1.3.6.1.4.1.14519.5.2.1.9999.103.1542239682251...,xr chest ap portable,"cr, dx"
4,1.3.6.1.4.1.14519.5.2.1.9999.103.2628412466675...,ct chest abdomen pelvis w,ct
...,...,...,...
9605,1.2.826.0.1.3680043.10.474.419639.330520289334...,xr chest 1 view ap,"cr, dx"
9606,1.2.826.0.1.3680043.10.474.419639.243984412691...,xr chest 1 view ap,"cr, dx"
9607,1.2.826.0.1.3680043.10.474.8868759358682592982...,[blank],"cr, dx"
9608,1.2.826.0.1.3680043.10.474.2373670981777278049...,[blank],"cr, dx"


In [11]:
## Check modalities for presence in the mapping table and frequency in MIDRC data:
study_modalities = list(set(studies.study_modality))
print("There are the following values of study_modality in the MIDRC imaging_study data: \n{}".format(study_modalities))

missing_modalities = set(study_modalities).difference(modalities)

mfreq = {}
for mod in missing_modalities:
    freq = len(studies.loc[studies['study_modality']==mod])
    mfreq[mod] = freq

print("\nThese modalities are in the study data but not in the mapping table: \n{}".format(mfreq))

There are the following values of study_modality in the MIDRC imaging_study data: 
['mr', 'cr, dx', 'ct', 'sr', 'ct,pt', 'ct,nm', 'ct,ot,pt']

These modalities are in the study data but not in the mapping table: 
{'ct,pt': 3, 'ct,nm': 5, 'ct,ot,pt': 2, 'sr': 2}


## Do mapping using pandas pd.merge(): 
---

1) Merge `loinc_code` in `mapping` dataframe into the `studies` dataframe on the combination of `study_description` and `study_modality`.

2) Merge the `loinc_long_common_name`, `loinc_method`, `loinc_contrast`, and `loinc_system` in `filters` dataframe into the `studies` dataframe on `loinc_code`.



In [12]:
sdf = studies.merge(mapping,on=['study_modality','study_description'],how='left')
sdf = sdf.merge(filters,on='loinc_code',how='left')
sdf.drop(columns=['study_description','study_modality'],inplace=True,errors='ignore')
sdf


,submitter_id,loinc_code,loinc_long_common_name,loinc_method,loinc_contrast,loinc_system
0,1.3.6.1.4.1.14519.5.2.1.9999.103.1217405707426...,24632-2,Portable XR Chest Views AP,XR.portable,NaN,Chest
1,1.3.6.1.4.1.14519.5.2.1.9999.103.8220481510500...,24632-2,Portable XR Chest Views AP,XR.portable,NaN,Chest
2,1.3.6.1.4.1.14519.5.2.1.9999.103.1753044623609...,24632-2,Portable XR Chest Views AP,XR.portable,NaN,Chest
3,1.3.6.1.4.1.14519.5.2.1.9999.103.1542239682251...,24632-2,Portable XR Chest Views AP,XR.portable,NaN,Chest
4,1.3.6.1.4.1.14519.5.2.1.9999.103.2628412466675...,72254-6,CT Chest and Abdomen and Pelvis W contrast IV,CT,W,Chest+Abdomen+Pelvis
...,...,...,...,...,...,...
9605,1.2.826.0.1.3680043.10.474.419639.330520289334...,36572-6,XR Chest AP,XR,NaN,Chest
9606,1.2.826.0.1.3680043.10.474.419639.243984412691...,36572-6,XR Chest AP,XR,NaN,Chest
9607,1.2.826.0.1.3680043.10.474.8868759358682592982...,43468-8,XR Unspecified body region Views,XR,NaN,Unspecified
9608,1.2.826.0.1.3680043.10.474.2373670981777278049...,43468-8,XR Unspecified body region Views,XR,NaN,Unspecified


In [13]:
## Pull in the original study_modality and study_description (non-lowercase) and rearrange the columns
cols = ['type','project_id','submitter_id','cases.submitter_id','study_modality','study_description','loinc_code','loinc_long_common_name','loinc_method','loinc_contrast','loinc_system']
sdf = sdf.merge(st[['type','project_id','submitter_id','cases.submitter_id','study_description','study_modality']], on='submitter_id',how='left')[cols]
sdf


,type,project_id,submitter_id,cases.submitter_id,study_modality,study_description,loinc_code,loinc_long_common_name,loinc_method,loinc_contrast,loinc_system
0,imaging_study,TCIA-COVID-19-AR,1.3.6.1.4.1.14519.5.2.1.9999.103.1217405707426...,COVID-19-AR-16406542,DX,XR CHEST AP PORTABLE,24632-2,Portable XR Chest Views AP,XR.portable,NaN,Chest
1,imaging_study,TCIA-COVID-19-AR,1.3.6.1.4.1.14519.5.2.1.9999.103.8220481510500...,COVID-19-AR-16424111,DX,XR CHEST AP PORTABLE,24632-2,Portable XR Chest Views AP,XR.portable,NaN,Chest
2,imaging_study,TCIA-COVID-19-AR,1.3.6.1.4.1.14519.5.2.1.9999.103.1753044623609...,COVID-19-AR-16434411,DX,XR CHEST AP PORTABLE,24632-2,Portable XR Chest Views AP,XR.portable,NaN,Chest
3,imaging_study,TCIA-COVID-19-AR,1.3.6.1.4.1.14519.5.2.1.9999.103.1542239682251...,COVID-19-AR-16434453,DX,XR CHEST AP PORTABLE,24632-2,Portable XR Chest Views AP,XR.portable,NaN,Chest
4,imaging_study,TCIA-COVID-19-AR,1.3.6.1.4.1.14519.5.2.1.9999.103.2628412466675...,COVID-19-AR-16424076,CT,CT CHEST ABDOMEN PELVIS W,72254-6,CT Chest and Abdomen and Pelvis W contrast IV,CT,W,Chest+Abdomen+Pelvis
...,...,...,...,...,...,...,...,...,...,...,...
9605,imaging_study,TCIA-RICORD,1.2.826.0.1.3680043.10.474.419639.330520289334...,419639-000771,DX,XR CHEST 1 VIEW AP,36572-6,XR Chest AP,XR,NaN,Chest
9606,imaging_study,TCIA-RICORD,1.2.826.0.1.3680043.10.474.419639.243984412691...,419639-000758,DX,XR CHEST 1 VIEW AP,36572-6,XR Chest AP,XR,NaN,Chest
9607,imaging_study,TCIA-RICORD,1.2.826.0.1.3680043.10.474.8868759358682592982...,SITE2-000268,CR,NaN,43468-8,XR Unspecified body region Views,XR,NaN,Unspecified
9608,imaging_study,TCIA-RICORD,1.2.826.0.1.3680043.10.474.2373670981777278049...,SITE2-000199,CR,NaN,43468-8,XR Unspecified body region Views,XR,NaN,Unspecified


In [14]:
mdf = sdf.loc[~sdf['loinc_code'].isna()]
udf = sdf.loc[sdf['loinc_code'].isna()]
fdf = udf.groupby(["study_modality", "study_description"]).size().reset_index(name="Freq").sort_values(by='Freq',ascending=False)
print("Total Studies: {}, Mapped: {}, Unmapped: {}".format(len(sdf),len(mdf),len(udf)))


Total Studies: 9610, Mapped: 9556, Unmapped: 54


In [15]:
## Save results to files
filename="{}/TCIA_LOINC_mapping_results_{}_{}.tsv".format(results_dir,len(sdf),today)
sdf.to_csv(filename, sep='\t', index=False)
print("All LOINC mapping results for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/TCIA_LOINC_mapped_{}_{}.tsv".format(results_dir,len(mdf),today)
mdf.to_csv(filename, sep='\t', index=False)
print("Successfully mapped imaging studies for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/TCIA_LOINC_umapped_{}_{}.tsv".format(results_dir,len(udf),today)
udf.to_csv(filename, sep='\t', index=False)
print("Unmapped imaging studies for projects {} saved to file: \n{}\n".format(projects,filename))

filename="{}/TCIA_LOINC_umapped_frequencies_{}_{}.tsv".format(results_dir,len(fdf),today)
fdf.to_csv(filename, sep='\t', index=False)
print("Frequencies of unmapped imaging study description/modality combinations for projects {} saved to file: \n{}\n".format(projects,filename))


All LOINC mapping results for projects ['TCIA-COVID-19-AR', 'TCIA-COVID-19-NY-SBU', 'TCIA-COVID-19_CT_Images', 'TCIA-RICORD'] saved to file: 
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-10/TCIA_LOINC_mapping_results_9610_2022-11-10.tsv

Successfully mapped imaging studies for projects ['TCIA-COVID-19-AR', 'TCIA-COVID-19-NY-SBU', 'TCIA-COVID-19_CT_Images', 'TCIA-RICORD'] saved to file: 
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-10/TCIA_LOINC_mapped_9556_2022-11-10.tsv

Unmapped imaging studies for projects ['TCIA-COVID-19-AR', 'TCIA-COVID-19-NY-SBU', 'TCIA-COVID-19_CT_Images', 'TCIA-RICORD'] saved to file: 
/Users/christopher/Documents/Notes/MIDRC/LOINC/results_2022-11-10/TCIA_LOINC_umapped_54_2022-11-10.tsv

Frequencies of unmapped imaging study description/modality combinations for projects ['TCIA-COVID-19-AR', 'TCIA-COVID-19-NY-SBU', 'TCIA-COVID-19_CT_Images', 'TCIA-RICORD'] saved to file: 
/Users/christopher/Documents/Notes/MIDRC/LOINC/results

In [16]:
## Take a glance at frequency of the unmapped imaging study modality/description combinations
display(fdf)


,study_modality,study_description,Freq
11,CT,"THORAX/ABDOMEN/PELVIS, CONT",14
9,CT,"THORAX PE, LOW DOSE",5
12,"CT,NM",LUNG PERFU SION IMAGING,3
10,CT,"THORAX, CONT",3
16,"CT,PT",PET CT FDG IMAG SKULL TO THIGH,3
21,MR,MRI ABD WWO IV CONT,2
19,DX,XR CHEST PA ONLY,2
18,DX,XR ACUTE ABDOMINAL SERIES W PA CHEST PORTABLE,2
15,"CT,OT,PT",PET CT FDG IMAG SKULL TO THIGH,2
1,CR,ABD UPRT VIEWONLY,2


## Submit the data to staging
---
Use the Gen3SDK function `Gen3Submission.submit_file()` to update the LOINC properties for each of the newly mapped studies in `mdf`.

In [ ]:
# Submit the derived data to staging
projects = list(set(mdf['project_id']))
data = {}
for pid in projects:
    print("Submitting data to project '{}'.".format(pid))
    data[pid] = sexp.submit_df(df=mdf.loc[mdf["project_id"]==pid],project_id=pid, chunk_size=1000)


Submitting data to project 'TCIA-RICORD'.
Submitting ['imaging_study'] DataFrame with 1216 records.
	Chunk 1 (chunk size: 1000, submitted: 0 of 1216)


/Users/christopher/Documents/Notes/MIDRC/LOINC/expansion.py:1969: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(


	 Succeeded: 1000 entities.
	Chunk 2 (chunk size: 1000, submitted: 1000 of 1216)
	 Succeeded: 216 entities.
Finished data submission.
Successful records: 1216
Failed invalid records: 0
Submitting data to project 'TCIA-COVID-19-AR'.
Submitting ['imaging_study'] DataFrame with 252 records.
	Chunk 1 (chunk size: 1000, submitted: 0 of 252)


/Users/christopher/Documents/Notes/MIDRC/LOINC/expansion.py:1969: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(


	 Succeeded: 252 entities.
Finished data submission.
Successful records: 252
Failed invalid records: 0
Submitting data to project 'TCIA-COVID-19-NY-SBU'.
Submitting ['imaging_study'] DataFrame with 7335 records.
	Chunk 1 (chunk size: 1000, submitted: 0 of 7335)


/Users/christopher/Documents/Notes/MIDRC/LOINC/expansion.py:1969: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(
